In [26]:
import pandas as pd
import time
import io
from api_request_funtions import ApiGet
from bq_transfers import BqDataTransfers 
from main import fetch_and_save
import json

In [23]:
thrivecart_get = ApiGet(http='thrivecart.com', api_key='TZ5TJYBR-FDB85IBI-0RFTB00N-VQ7ZFY2S')
thrivecart_save = BqDataTransfers(gcp_project_id= 'arboreal-cat-451816-n0', bq_data_set = 'thrive_cart')
bq_client = thrivecart_save.get_bq_client("/Users/shami/Library/Mobile Documents/com~apple~CloudDocs/Personal Projects/vexis/vexis_bq_writter.json")

In [28]:
# def fetch_transactions(event, context):
    # Fetch the total count of transactions from the API
x = thrivecart_get.fetch_data(end_point=f'/api/external/transactions?page=0&perPage=0&query=&transactionType=any')
total_records_api = x['meta']['total']
total_records_bq = bq_client.get_table('thrive_cart.transactions').num_rows

# Calculate the difference and determine the number of pages to fetch
records_to_fetch = max(0, total_records_api - total_records_bq)
pages = list(range(1, (records_to_fetch // 100) + 2)) if records_to_fetch > 0 else []

# Fetch new transactions
for page in pages:
    x = thrivecart_get.fetch_data(
        end_point=f'/api/external/transactions?page={page}&perPage=100&query=&transactionType=any'
    )
    transactions = x.get("transactions", [])

    if transactions:
        # Convert to NDJSON format
        ndjson_lines = "\n".join(json.dumps(transaction) for transaction in transactions)
        ndjson_buffer = io.StringIO(ndjson_lines)

        # Upload to BigQuery, ensuring no duplicates based on transaction_id
        thrivecart_save.start_transfer_json(
            bq_client=bq_client,
            file=ndjson_buffer,
            destination_table='transactions',
            write_options='append'
        )

    time.sleep(0.5)


BadRequest: 400 Provided Schema does not match Table arboreal-cat-451816-n0:thrive_cart.transactions. Field internal_subscription_id has changed type from INTEGER to STRING; reason: invalid, message: Provided Schema does not match Table arboreal-cat-451816-n0:thrive_cart.transactions. Field internal_subscription_id has changed type from INTEGER to STRING